# Extract tables

Given a cohort selection query, extract selected tables for patients in cohort

Relevant tables:
- CHARTEVENTS
- LABEVENTS
- PATIENTS

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import os

In [ ]:
output_dir = '/Users/jk1/temp/mimic/extraction'

In [ ]:
%matplotlib inline
plt.style.use('ggplot')

# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser)

# the below statement is prepended to queries to ensure they select from the right schema
query_schema = 'set search_path to ' + schema_name + ';'

In [ ]:
selection_query_path = './patient_selection_query.sql'
# load in the text of the query
with open(selection_query_path) as fp:
    patient_selection_query = ''.join(fp.readlines())

# Extract mortality outcome

- in-hospital mortality (needs dod, hospital-discharge date)
- 6m mortality (needs dod, hospital admission date)

In [ ]:
query = query_schema + patient_selection_query + """

SELECT selection.subject_id, selection.hadm_id, selection.icustay_id  , selection.age,
  admissions.admittime, admissions.dischtime, pat.dod

FROM selection

INNER JOIN patients pat
  ON selection.subject_id = pat.subject_id

INNER JOIN admissions admissions
  ON selection.hadm_id = admissions.hadm_id

WHERE selection.exclusion_discharge_diagnosis = 0
    AND selection.exclusion_first_stay = 0
    AND selection.exclusion_age = 0
    AND selection.exclusion_los = 0
    AND selection.exclusion_non_urgent = 0
    AND selection.exclusion_admission_diagnosis = 0
"""
mortality_df = pd.read_sql_query(query, con)
mortality_df

In [ ]:
mortality_df.to_csv(os.path.join(output_dir, 'mortality_df.csv'))

# Extract lab df

In [ ]:
query = query_schema + """
WITH sel_lab as
(
""" + patient_selection_query + """
SELECT selection.subject_id, selection.hadm_id, selection.icustay_id,
        lab.itemid, lab.charttime,	lab.value,	lab.valuenum,	lab.valueuom
FROM selection

INNER JOIN labevents as lab
    on selection.hadm_id = lab.hadm_id

WHERE selection.exclusion_discharge_diagnosis = 0
    AND selection.exclusion_first_stay = 0
    AND selection.exclusion_age = 0
    AND selection.exclusion_los = 0
)

SELECT sel_lab.subject_id, sel_lab.hadm_id, sel_lab.icustay_id,
        sel_lab.itemid, d_labitems.label,	sel_lab.charttime,	sel_lab.value,	sel_lab.valuenum,	sel_lab.valueuom
FROM sel_lab
LEFT JOIN d_labitems as d_labitems
    on sel_lab.itemid = d_labitems.itemid

"""
lab_df = pd.read_sql_query(query, con)
lab_df

In [ ]:
lab_df.to_csv(os.path.join(output_dir, 'lab_df.csv'))

# Extract monitoring

In [ ]:
query = query_schema + """
WITH sel_monitoring as
(
""" + patient_selection_query + """
SELECT selection.subject_id, selection.hadm_id, selection.icustay_id,
        chart.itemid,	chart.charttime,	chart.storetime,	chart.value,	chart.valuenum,	chart.valueuom

FROM selection

INNER JOIN chartevents as chart
    on selection.hadm_id = chart.hadm_id

WHERE selection.exclusion_discharge_diagnosis = 0
    AND selection.exclusion_first_stay = 0
    AND selection.exclusion_age = 0
    AND selection.exclusion_los = 0
)

SELECT sel_monitoring.subject_id, sel_monitoring.hadm_id, sel_monitoring.icustay_id,
    sel_monitoring.itemid, d_items.label,	sel_monitoring.charttime,	sel_monitoring.storetime,	sel_monitoring.value,	sel_monitoring.valuenum,	sel_monitoring.valueuom
FROM sel_monitoring
LEFT JOIN d_items as d_items
    on sel_monitoring.itemid = d_items.itemid


"""
monitoring_df = pd.read_sql_query(query, con)
monitoring_df

In [ ]:
monitoring_df.to_csv(os.path.join(output_dir, 'monitoring_df.csv'))

# Admission variables

In [ ]:
query = query_schema + """
WITH sel_admission as
(
""" + patient_selection_query + """
SELECT selection.subject_id, selection.hadm_id, selection.icustay_id, selection.age, pat.gender, admissions.diagnosis, admissions.admission_type, admissions.admission_location, chart.itemid, chart.value as chart_value

FROM selection

INNER JOIN patients pat
  ON selection.subject_id = pat.subject_id

INNER JOIN admissions admissions
  ON selection.hadm_id = admissions.hadm_id

LEFT JOIN chartevents as chart
    ON selection.hadm_id = chart.hadm_id
    AND (chart.itemid = 225059 OR chart.itemid = 225811)

WHERE selection.exclusion_discharge_diagnosis = 0
    AND selection.exclusion_first_stay = 0
    AND selection.exclusion_age = 0
    AND selection.exclusion_los = 0
)

SELECT sel_admission.subject_id, sel_admission.hadm_id, sel_admission.icustay_id,
    sel_admission.age, sel_admission.gender, sel_admission.admission_type, sel_admission.diagnosis, sel_admission.admission_location,
    sel_admission.itemid, d_items.label, sel_admission.chart_value

FROM sel_admission
LEFT JOIN d_items as d_items
    on sel_admission.itemid = d_items.itemid


"""
admission_df = pd.read_sql_query(query, con)
admission_df

In [ ]:
admission_df.to_csv(os.path.join(output_dir, 'admission_df.csv'))

# Procedures variables

In [ ]:
query = query_schema + """
WITH sel_procedure as
(
""" + patient_selection_query + """
SELECT selection.subject_id, selection.hadm_id, selection.icustay_id,
        procedures.icd9_code, in_cv.itemid as citemid, in_mv.itemid as mitemid

FROM selection

LEFT JOIN procedures_icd as procedures
  ON selection.hadm_id = procedures.hadm_id
  AND (procedures.icd9_code::integer = 3974 OR procedures.icd9_code::integer = 9910)

LEFT JOIN inputevents_cv as in_cv
    ON selection.hadm_id = in_cv.hadm_id
    AND (in_cv.itemid = 227056 OR in_cv.itemid = 2666 OR in_cv.itemid = 2667 OR in_cv.itemid = 42575 OR in_cv.itemid = 221319)

LEFT JOIN inputevents_mv as in_mv
    ON selection.hadm_id = in_mv.hadm_id
    AND (in_mv.itemid = 227056 OR in_mv.itemid = 2666 OR in_mv.itemid = 2667 OR in_mv.itemid = 42575 OR in_mv.itemid = 221319)


WHERE selection.exclusion_discharge_diagnosis = 0
    AND selection.exclusion_first_stay = 0
    AND selection.exclusion_age = 0
    AND selection.exclusion_los = 0
)

SELECT sel_procedure.subject_id, sel_procedure.hadm_id, sel_procedure.icustay_id,
    sel_procedure.icd9_code, d_icd_procedures.long_title, d_items.label

FROM sel_procedure

LEFT JOIN d_icd_procedures
    on sel_procedure.icd9_code = d_icd_procedures.icd9_code

LEFT JOIN d_items as d_items
    ON sel_procedure.citemid = d_items.itemid
    OR sel_procedure.mitemid = d_items.itemid

"""
procedure_df = pd.read_sql_query(query, con)
procedure_df

In [ ]:
procedure_df.to_csv(os.path.join(output_dir, 'procedure_df.csv'))

In [ ]:
query = query_schema + """
WITH sel_admission as
(
""" + patient_selection_query + """
SELECT selection.subject_id, selection.hadm_id, selection.icustay_id, selection.age, pat.gender, admissions.diagnosis, admissions.admission_type, admissions.admission_location, chart.itemid, chart.value as chart_value

FROM selection

INNER JOIN patients pat
  ON selection.subject_id = pat.subject_id

INNER JOIN admissions admissions
  ON selection.hadm_id = admissions.hadm_id

INNER JOIN chartevents as chart
    ON selection.hadm_id = chart.hadm_id
    AND (chart.itemid = 225059 OR chart.itemid = 225811)

WHERE selection.exclusion_discharge_diagnosis = 0
    AND selection.exclusion_first_stay = 0
    AND selection.exclusion_age = 0
    AND selection.exclusion_los = 0
)

SELECT sel_admission.subject_id, sel_admission.hadm_id, sel_admission.icustay_id,
    sel_admission.age, sel_admission.gender, sel_admission.admission_type, sel_admission.diagnosis, sel_admission.admission_location,
    sel_admission.itemid, d_items.label, sel_admission.chart_value

FROM sel_admission
LEFT JOIN d_items as d_items
    on sel_admission.itemid = d_items.itemid


"""
admission_df = pd.read_sql_query(query, con)
admission_df

In [ ]:
admission_df.to_csv(os.path.join(output_dir, 'admission_df.csv'))

# Procedures variables

In [ ]:
query = query_schema + """
WITH sel_procedure as
(
""" + patient_selection_query + """
SELECT selection.subject_id, selection.hadm_id, selection.icustay_id,
        procedures.icd9_code, in_cv.itemid as citemid, in_mv.itemid as mitemid

FROM selection

LEFT JOIN procedures_icd as procedures
  ON selection.hadm_id = procedures.hadm_id
  AND (procedures.icd9_code::integer = 3974 OR procedures.icd9_code::integer = 9910)

LEFT JOIN inputevents_cv as in_cv
    ON selection.hadm_id = in_cv.hadm_id
    AND (in_cv.itemid = 227056 OR in_cv.itemid = 2666 OR in_cv.itemid = 2667 OR in_cv.itemid = 42575 OR in_cv.itemid = 221319)

LEFT JOIN inputevents_mv as in_mv
    ON selection.hadm_id = in_mv.hadm_id
    AND (in_mv.itemid = 227056 OR in_mv.itemid = 2666 OR in_mv.itemid = 2667 OR in_mv.itemid = 42575 OR in_mv.itemid = 221319)


WHERE selection.exclusion_discharge_diagnosis = 0
    AND selection.exclusion_first_stay = 0
    AND selection.exclusion_age = 0
    AND selection.exclusion_los = 0
)

SELECT sel_procedure.subject_id, sel_procedure.hadm_id, sel_procedure.icustay_id,
    sel_procedure.icd9_code, d_icd_procedures.long_title, d_items.label

FROM sel_procedure

LEFT JOIN d_icd_procedures
    on sel_procedure.icd9_code = d_icd_procedures.icd9_code

LEFT JOIN d_items as d_items
    ON sel_procedure.citemid = d_items.itemid
    OR sel_procedure.mitemid = d_items.itemid

"""
procedure_df = pd.read_sql_query(query, con)
procedure_df

In [ ]:
procedure_df.to_csv(os.path.join(output_dir, 'procedure_df.csv'))